# Image Analysis & Comparison

This notebook analyzes `coin.jpg` and `varroe.jpg` to extract EXIF metadata and internal JPEG algorithm information (Quantization Tables).

In [1]:

import PIL.Image
from PIL.ExifTags import TAGS
import pandas as pd
import os
from IPython.display import display

def get_exif_data(image):
    'Extracts EXIF data from a PIL image.'
    exif_data = {}
    # getexif() returns the raw EXIF data
    raw_exif = image.getexif()
    if not raw_exif:
        return exif_data
        
    for tag_id, value in raw_exif.items():
        tag = TAGS.get(tag_id, tag_id)
        # Decode bytes if necessary for readability
        if isinstance(value, bytes):
            try:
                value = value.decode()
            except:
                value = str(value)
        exif_data[tag] = value
    return exif_data

def get_quantization_tables(image):
    'Extracts JPEG quantization tables.'
    if not hasattr(image, 'quantization'):
        return "No Quantization Tables (Not a JPEG?)"
    
    tables = {}
    # quantization is a dictionary where keys are table indices (0 for luminance, 1 for chrominance)
    # The values are usually lists or bytes
    if image.quantization:
        for key, table in image.quantization.items():
            tables[f'Table {key}'] = list(table) if hasattr(table, '__iter__') else table
    return tables

def analyze_image(filepath):
    if not os.path.exists(filepath):
        return None
        
    try:
        with PIL.Image.open(filepath) as img:
            info = {
                'Filename': os.path.basename(filepath),
                'Format': img.format,
                'Mode': img.mode,
                'Size': str(img.size),
            }
            
            # Get EXIF
            exif = get_exif_data(img)
            # Flatten EXIF into info with prefix
            for k, v in exif.items():
                info[f'EXIF: {k}'] = v
                
            # Get Quantization info
            quant = get_quantization_tables(img)
            info['Quantization'] = quant
            
            return info
    except Exception as e:
        return {'Filename': os.path.basename(filepath), 'Error': str(e)}

files = ['coin.jpg', 'varroe.jpg']
results = [analyze_image(f) for f in files]
results = [r for r in results if r] # Filter None

# Convert to DataFrame for easier comparison
df = pd.DataFrame(results).T
if not df.empty:
    df.columns = [r['Filename'] for r in results]
    print("--- Basic & EXIF Comparison ---")
    # Display everything except complex nested objects like Quantization first
    display(df.drop('Quantization', errors='ignore'))
else:
    print("No data found or images could not be loaded.")

print("\n--- Quantization Tables (Compression Fingerprint) ---")
for res in results:
    print(f"\nImage: {res['Filename']}")
    q_tables = res.get('Quantization')
    if isinstance(q_tables, dict):
        for t_name, table in q_tables.items():
            # Print first few values to keep it readable, but you can print whole table
            print(f"  {t_name}: {str(table)[:50]}... (Total {len(table) if hasattr(table, '__len__') else 0} values)")
    else:
        print(f"  {q_tables}")


--- Basic & EXIF Comparison ---


/usr/local/lib/python3.11/site-packages/PIL/Image.py:3451: DecompressionBombWarning: Image size (108576768 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


,coin.jpg,varroe.jpg
Filename,coin.jpg,varroe.jpg
Format,JPEG,JPEG
Mode,RGB,RGB
Size,"(9024, 12032)","(12032, 9024)"
EXIF: ImageWidth,9024,12032
EXIF: ImageLength,12032,9024
EXIF: GPSInfo,889,889
EXIF: ResolutionUnit,2,2
EXIF: ExifOffset,208,208
EXIF: Make,OPPO,OPPO



--- Quantization Tables (Compression Fingerprint) ---

Image: coin.jpg
  Table 0: [2, 1, 1, 2, 2, 4, 5, 6, 1, 1, 1, 2, 3, 6, 6, 6, 1... (Total 64 values)
  Table 1: [2, 2, 2, 5, 10, 10, 10, 10, 2, 2, 3, 7, 10, 10, 1... (Total 64 values)

Image: varroe.jpg
  Table 0: [2, 1, 1, 2, 2, 4, 5, 6, 1, 1, 1, 2, 3, 6, 6, 6, 1... (Total 64 values)
  Table 1: [2, 2, 2, 5, 10, 10, 10, 10, 2, 2, 3, 7, 10, 10, 1... (Total 64 values)
